In [3]:
#直接读取PDF文件内容，并保存到txt文件中。我们不用

from io import StringIO
from io import open
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
 
def read_pdf(pdf):
    # resource manager
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    # device
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    PDFPage.get_pages(rsrcmgr, device, pdf)
    device.close()
    content = retstr.getvalue()
    retstr.close()
    # 获取所有行
    lines = str(content).split("\n")
    return lines
 
with open('./temp/temp.pdf', "rb") as my_pdf:
    with open('result.txt','w+') as f:
        for x in read_pdf(my_pdf):
            f.write(x)
            print(x)

In [2]:
#直接读取pdf文件内容，并打印出来。我们不用

import PyPDF2

def Read(startPage, endPage):
    text = []
    cleanText = ""
    try:
        pdfFileObj = open('./temp/temp.pdf', 'rb')
        pdfReader = PyPDF2.PdfReader(pdfFileObj)
        num_pages = len(pdfReader.pages)
        print(f"Total number of pages: {num_pages}")
        
        if endPage >= num_pages:
            print(f"End page {endPage} is out of range. Total pages are {num_pages}.")
            return
        
        while startPage <= endPage:
            pageObj = pdfReader.pages[startPage]
            page_text = pageObj.extract_text()
            if page_text:
                text.append(page_text)
            startPage += 1
        
        pdfFileObj.close()
        
        for page_text in text:
            cleanText += page_text + "\n"
        
        cleanText = cleanText.strip()
        print(cleanText)
    
    except KeyboardInterrupt:
        print("Program interrupted by user.")
    except Exception as e:
        print(f"An error occurred: {e}")

Read(25, 35)


Total number of pages: 197



In [ ]:
#这是写的切割的临时代码块。不用看。
import os
import fitz  # PyMuPDF
from PyPDF2 import PdfWriter, PdfReader
import glob

PDF_file_path = r'./Part3/p3_3.pdf'
output_file_path = r'./temp'
max_pages = 750
# doc = fitz.open(PDF_file_path)

def clear_pdf_files(directory):
    # 获取目录下所有PDF文件的路径
    if os.path.exists(directory) == False:
        os.mkdir(directory)
    pdf_files = glob.glob(os.path.join(directory, '*.pdf'))
    
    # 删除每个PDF文件
    for pdf_file in pdf_files:
        try:
            os.remove(pdf_file)
            print(f"Deleted {pdf_file}")
        except Exception as e:
            print(f"Error deleting {pdf_file}: {e}")

clear_pdf_files(output_file_path)

def split_pdf(input_path, max_pages, output_prefix):
    # 打开PDF文件
    pdf_document = fitz.open(input_path)
    total_pages = len(pdf_document)

    if total_pages <= max_pages:
        print(f"PDF has {total_pages} pages, no need to split.")
        output_path = os.path.join(output_prefix, "temp.pdf")
        output_pdf = fitz.open()
        output_pdf.insert_pdf(pdf_document, from_page=0, to_page=total_pages)
        output_pdf.save(output_path)
        output_pdf.close()
        return

    # 计算需要分割的份数
    num_parts = (total_pages + max_pages - 1) // max_pages

    for part in range(num_parts):
        output_path = os.path.join(output_prefix, f"temp_part_{part + 1}.pdf")
        output_pdf = fitz.open()

        start_page = part * max_pages
        end_page = min(start_page + max_pages, total_pages)

        for page_num in range(start_page, end_page):
            output_pdf.insert_pdf(pdf_document, from_page=page_num, to_page=page_num)

        output_pdf.save(output_path)
        output_pdf.close()

        print(f"Created {output_path} with pages {start_page + 1} to {end_page}")

split_pdf(PDF_file_path, max_pages, output_file_path)

In [ ]:
#查看ORC识别时是否有效去除页眉页脚。每一个新的PDF都用这个来看裁剪的参数

import fitz
import time
import os
from paddleocr import PaddleOCR
import numpy as np
from IPython.display import display
import ipywidgets as widgets

ocr = PaddleOCR(det=False, use_gpu=False, enable_mkldnn=True, use_tensorrt=True, use_angle_cls=True, lang='ch')

def pdf_to_jpg(name):
    pdfdoc = fitz.open(name)
    temp = 0
    page_index = 0
    
    def display_next_image(b):
        nonlocal page_index
        if page_index < pdfdoc.page_count:
            page = pdfdoc[page_index]
            rotate = int(0)
            rect = page.rect

            #裁剪范围
            crop_top = 50
            crop_bottom = 30
            crop_right = 40
            crop_left = 40

            #缩放比例
            zoom_x = 0.8
            zoom_y = 0.8
            rotate = int(0)
            trans = fitz.Matrix(zoom_x, zoom_y).prerotate(rotate)
            crop = fitz.Rect(rect.x0 + crop_left, rect.y0 + crop_top, rect.x1 - crop_right, rect.y1 - crop_bottom)
            trans = fitz.Matrix(zoom_x, zoom_y).prerotate(rotate)
            pm_cropped = page.get_pixmap(matrix=trans, alpha=False, clip=crop)
            pm_original = page.get_pixmap(matrix=trans, alpha=False)
            
            # 将 pixmap 转换为图像并显示
            img_cropped = widgets.Image(value=pm_cropped.tobytes(), format='png')
            img_original = widgets.Image(value=pm_original.tobytes(), format='png')
            
            output.clear_output(wait=True)
            with output:
                display(widgets.HBox([img_original, img_cropped]))
            page_index += 1
        else:
            output.clear_output(wait=True)
            with output:
                print("所有图片已显示完毕。")
    
    button = widgets.Button(description="显示下一张图片")
    output = widgets.Output()
    button.on_click(display_next_image)
    
    display(button, output)

pdf_to_jpg(r'./Part3/P3_4.pdf')


[2024/11/09 00:53:43] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=True, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\28407/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\28407/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_leng

Button(description='显示下一张图片', style=ButtonStyle())

Output()

In [ ]:
#使用OCR技术，将PDF文件转化为文本文件，并且消除页眉页脚内容，现在暂定最佳的PDF读取方法。但我们不用

import fitz
pdf_file = "./test4.pdf"
pdf_document = fitz.open(pdf_file)
text = ""
for page_number in range(len(pdf_document)):
    page = pdf_document.load_page(page_number)
    rect = page.rect

    #需要从上一个代码块得到参数
    clip_x = 40
    clip_y = 85

    zoom_x = 16.0
    zoom_y =16.0
    rotate = int(0)
    trans = fitz.Matrix(zoom_x, zoom_y).prerotate(rotate)
    crop = fitz.Rect(clip_x, clip_y, rect.width-clip_x, rect.height-clip_y)
    # pm = page.get_pixmap(matrix=trans, alpha=False, clip = crop)
    # pm._writeIMG('temp.jpg', 1, jpg_quality = 95)
    # input()
    for block in page.get_text("blocks", clip = crop):
        x0, y0, x1, y1 = block[0:4]
        text_block = block[4]
        # 根据文本块属性过滤表格中的文本
        # 这只是一个示例，你可以根据文本块的位置和其他属性来进一步过滤
        if y1 - y0 < 20:  # 通过高度过滤小文本块
            continue
        if "image" in text_block:
            continue
        text += text_block
with open(pdf_file.split('.pdf')[0] + '.txt', 'w+') as w:
    w.write(text)
pdf_document.close()
print(text)


In [ ]:
#pytesseract使用OCR图像识别来转换PDF文件中的文字内容。我们不用
import pytesseract
from PIL import Image
import os
from pdf2image import convert_from_path
from io import BytesIO

pytesseract.pytesseract.tesseract_cmd = r'D:\\Program Files\\Tesseract-OCR\\tesseract.exe'  # 根据你的安装路径调整
testdata_dir_config = '--tessdata-dir "D:\\Program Files\\Tesseract-OCR\\tessdata"'

if os.path.exists("result.txt"):
    os.remove("result.txt")
fo = open("result.txt", "w")

pages = convert_from_path("./problem/p1_4.pdf")

# 定义裁剪区域，例如去除页眉和页脚各50像素
crop_top = 110
crop_bottom = 60
crop_right = 40
crop_left = 40

for i, page in enumerate(pages):
    buf = BytesIO()
    page.save(buf, format="JPEG")
    buf.seek(0)
    img_page = Image.open(buf)
    
    # 获取图像的宽度和高度
    width, height = img_page.size
    
    # 裁剪图像，去除页眉和页脚
    img_cropped = img_page.crop((crop_left, crop_top, width - crop_right, height - crop_bottom))
    
    txt = pytesseract.image_to_string(img_cropped, lang='chi_sim')
    fo.write(txt + "\n")
    print(f"No {i} page result: {txt}")

fo.close()


In [2]:
# 针对问答式文本的文本格式调整
import re

pattern = r'\n\s{2}\d+\.\s{1}'

# 查找所有匹配的位置
matches = [(match.start(), match.end()) for match in re.finditer(pattern, text)]

def write_file(text, start, end):
    with open('./text3_modify.txt', 'a') as f:
        f.write(text[start:end] + '\r\n')

# 打印结果
for i in range(len(matches) - 1):
    start, end = matches[i]
    next_start, next_end = matches[i + 1]
    
    # 找到下一个换行符的位置
    next_newline = end
    while True:
        next_newline = text.find('\n', next_newline)
        if next_newline == -1:
            break
        # 检查换行符前是否有问号或句号
        if text[next_newline - 2] in '?.':
            break
        next_newline += 1

    if next_newline != -1:
        print(f"位置: {start} 到 {next_newline}, 内容: {text[start:next_newline]}")
        write_file(text, start, next_newline)
    else:
        print(f"位置: {start} 到 {end}, 内容: {text[start:end]}")
        write_file(text, start, end)
    
    # 打印两个问题之间的回答
    if next_start > next_newline:
        print(f"位置: {next_newline} 到 {next_start}, 内容: {text[next_newline:next_start]}")
        write_file(text, next_newline, next_start)

# 处理最后一个匹配
start, end = matches[-1]
next_newline = end
while True:
    next_newline = text.find('\n', next_newline)
    if next_newline == -1:
        break
    # 检查换行符前是否有问号或句号
    if text[next_newline - 2] in '?.':
        break
    next_newline += 1

if next_newline != -1:
    print(f"位置: {start} 到 {next_newline}, 内容: {text[start:next_newline]}")
    write_file(text, start, next_newline)
else:
    print(f"位置: {start} 到 {end}, 内容: {text[start:end]}")
    write_file(text, start, end)


NameError: name 'text' is not defined

In [1]:
# 使用LlamaParse库进行pdf转txt,每个账号每天只能跑1000页。

import os
import fitz  # PyMuPDF
import nest_asyncio
# from llama_parse import LlamaParse
from PyPDF2 import PdfWriter, PdfReader
import glob

# 定义裁剪区域，例如去除页眉和页脚各50像素。数值可以是负数。只要能跑就行

crop_top = 50
crop_bottom = 30
crop_right = 40
crop_left = 40
shift_y = 20
shift_x = 50

# 这个是提高清晰度的。一般不要小于1。如果pdf字体过小可以适度同比增大x和y的缩放值。
zoom_x = 2.0
zoom_y = 2.0
rotate = int(0)

PDF_file_path = r'./Part3/P3_13.pdf'
#所有的中间产物文件存放的位置
output_file_path = r'./temp'
output_txt_path = r'./chunks'
max_pages = 750
# doc = fitz.open(PDF_file_path)

#现在我们所有的临时文件都存在./temp目录下，这个是用来每次清理的。
def clear_pdf_files(directory):
    # 获取目录下所有PDF文件的路径
    if os.path.exists(directory) == False:
        os.mkdir(directory)
    pdf_files = glob.glob(os.path.join(directory, '*.pdf'))
    
    # 删除每个PDF文件
    for pdf_file in pdf_files:
        try:
            os.remove(pdf_file)
            print(f"Deleted {pdf_file}")
        except Exception as e:
            print(f"Error deleting {pdf_file}: {e}")

clear_pdf_files(output_file_path)

#剪切PDF文件，一份最大750页。分割后不推荐一次传输。因为一个api的上限就是1k。可以分次上传后再手动合并。
def split_pdf(input_path, max_pages, output_prefix):

    pdf_document = fitz.open(input_path)
    total_pages = len(pdf_document)

    if total_pages <= max_pages:
        print(f"PDF has {total_pages} pages, no need to split.")
        output_path = os.path.join(output_prefix, "temp.pdf")
        output_pdf = fitz.open()
        output_pdf.insert_pdf(pdf_document, from_page=0, to_page=total_pages)
        output_pdf.save(output_path)
        output_pdf.close()
        return

    # 计算需要分割的份数
    num_parts = (total_pages + max_pages - 1) // max_pages

    for part in range(num_parts):
        output_path = os.path.join(output_prefix, f"temp_part_{part + 1}.pdf")
        output_pdf = fitz.open()

        start_page = part * max_pages
        end_page = min(start_page + max_pages, total_pages)

        for page_num in range(start_page, end_page):
            output_pdf.insert_pdf(pdf_document, from_page=page_num, to_page=page_num)

        output_pdf.save(output_path)
        output_pdf.close()

        print(f"Created {output_path} with pages {start_page + 1} to {end_page}")

#新的剪切pdf的方法。如果前面的无法剪切报错cropbox不在mediabox内时可以改用这个。这个产出的pdf文件大小特别大。现在最好先不要用了。。。
def crop_pdf_new(input_pdf, output_pdf, crop_left, crop_top, crop_right, crop_bottom):

    doc = fitz.open(input_pdf)
    
    new_doc = fitz.open()

    #这个值不要太大，最多就到2吧。因为这个方法是直接保存图片的，每个pdf都特别大，zoom过大pdf会过大，上传会出现问题。
    zoom_x = 2.0  # 你可以根据需要调整这个值
    zoom_y = 2.0  # 你可以根据需要调整这个值
    matrix = fitz.Matrix(zoom_x, zoom_y)

    for page in doc:
        rect = page.rect
        

        clip = fitz.Rect(
            rect.x0 + crop_left,
            rect.y0 + crop_top,
            rect.x1 - crop_right,
            rect.y1 - crop_bottom
        )
        
        pix = page.get_pixmap(matrix = matrix, clip=clip)
        pix.set_dpi(150, 150)  # 设置DPI，降低分辨率
        
        new_page = new_doc.new_page(width=pix.width, height=pix.height)
        
        new_page.insert_image(new_page.rect, pixmap=pix)
    
    new_doc.save(output_pdf)

#另一个新的剪切pdf的方法。和旧方法类似。
def crop_pdf_pypdf2(input_pdf, output_pdf, crop_left, crop_top, crop_right, crop_bottom, shift_x, shift_y):
    with open(input_pdf, 'rb') as file:
        reader = PdfReader(file)
        writer = PdfWriter()

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            page.mediabox.lower_left  = (crop_left + shift_x, crop_bottom + shift_y)
            page.mediabox.lower_right  = (page.mediabox.width - crop_right + shift_x, crop_bottom + shift_y)
            page.mediabox.upper_left = (crop_left + shift_x, page.mediabox.height - crop_top + shift_y)
            page.mediabox.upper_Right = (page.mediabox.width - crop_right + shift_x, page.mediabox.height - crop_top + shift_y)
            writer.add_page(page)

        with open(output_pdf, 'wb') as output_file:
            writer.write(output_file)

#旧的剪切pdf的方法。
def crop_pdf(input_pdf, output_pdf, crop_left, crop_top, crop_right, crop_bottom):
    doc = fitz.open(input_pdf)
    for n, page in enumerate(doc):
        rect = page.rect
        media_box = page.mediabox
        # clip.x0 = max(clip.x0, media_box.x0)
        # clip.y0 = max(clip.y0, media_box.y0)
        # clip.x1 = min(clip.x1, media_box.x1)
        # clip.y1 = min(clip.y1, media_box.y1)

        # print(rect.x0, rect.y0, rect.x1, rect.y1)
        # print(media_box.x0, media_box.y0, media_box.x1, media_box.y1)
        if rect.x1 - rect.x0 < 10 or rect.y1 - rect.y0 < 10:
            print(f"Cannot read page {n+1}! Check it manually.")
            continue
        trans = fitz.Matrix(zoom_x, zoom_y).prerotate(rotate)
        clip = fitz.Rect(max(rect.x0 + crop_left, rect.x0), \
            max(rect.y0 + crop_top, rect.y0), \
                min(rect.x1 - crop_right, rect.x1), \
                    min(rect.y1 - crop_bottom, rect.y1))

        # print(clip.x0, clip.y0, clip.x1, clip.y1)

        page.set_cropbox(clip)

        doc.save(output_pdf)

nest_asyncio.apply()

# API_KEY = 'llx-YWsF0qbwjBsQlhd5RVPqjivhSiRR69FZKEpg36sbxQnGU9vF'
# reference: https://cloud.llamaindex.ai/parse

# parser = LlamaParse(
#     api_key=API_KEY,  # can also be set in your env as LLAMA_CLOUD_API_KEY
#     result_type="markdown",  # "markdown" and "text" are available
#     num_workers=4,  # if multiple files passed, split in `num_workers` API calls
#     verbose=True,
#     language="en",  # Optionally you can define a language, default=en
# )

temp_pdf_path = os.path.join(os.path.join(output_file_path, "temp.pdf"))

# cropped_pdf_doc = crop_pdf(PDF_file_path, temp_pdf_path, crop_left, crop_top, crop_right, crop_bottom)
# crop_pdf_pypdf2(PDF_file_path, temp_pdf_path, crop_left, crop_top, crop_right, crop_bottom, shift_x, shift_y)
crop_pdf_new(PDF_file_path, temp_pdf_path, crop_left, crop_top, crop_right, crop_bottom)

# split_pdf(temp_pdf_path, max_pages, output_file_path)

# documents = parser.load_data(temp_pdf_path)

# # 处理返回的文档
# with open(os.path.join(output_txt_path, PDF_file_path.split('.pdf')[0].split('/')[-1] + '_cropped.txt'), 'w+') as w:
    # for i in documents:
        # w.write(i.get_text())


In [2]:
from llama_parse import LlamaParse
import nest_asyncio

nest_asyncio.apply()
API_KEY = 'llx-diPwN1V1iqNH3eHS7IbDDZmNHLYYjbp067PB0K0mtNKEv2os'
# reference: https://cloud.llamaindex.ai/parse

pdf_file = './temp/temp.pdf'

parser = LlamaParse(
    api_key=API_KEY,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="en",  # Optionally you can define a language, default=en
)
documents = parser.load_data(pdf_file)

#处理返回的文档
with open('./chunks/P3_13_cropped.txt', 'w+') as w:
    for i in documents:
        w.write(i.get_text())

Started parsing the file under job_id 08f038e5-24de-4503-9e84-467f593495e1
...................

In [ ]:
txt_file_name = './test3.txt'
txt_content = ''
with open(txt_file_name, 'r') as f:
    for line in f:
        txt_content += line.replace('\n', '')
print(txt_content)


In [ ]:
import nltk
# nltk.download('punkt')

# Splitting Text into Sentences
def split_text_into_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

sentences = split_text_into_sentences(txt_content)

In [ ]:
output_file = './text3_chunks.txt'

with open(output_file, 'w') as o:
    for sentence in final_texts:
        o.write(sentence + '\r\n')
        print(sentence)

In [ ]:
#Kmeans结合Spacy提取文本特征并进行文本聚类切chunks，现在采用的切chunk的方法
import numpy as np
import spacy

min_token_length = 150
max_token_length = 400


# Load the Spacy model
nlp = spacy.load('en_core_web_sm')

def process(text):
    doc = nlp(text)
    sents = list(doc.sents)
    vecs = np.stack([sent.vector / sent.vector_norm for sent in sents])

    return sents, vecs

def cluster_text(sents, vecs, threshold):
    clusters = [[0]]
    for i in range(1, len(sents)):
        if np.dot(vecs[i], vecs[i-1]) < threshold:
            clusters.append([])
        clusters[-1].append(i)
    
    return clusters

def clean_text(text):
    # Add your text cleaning process here
    return text

# Initialize the clusters lengths list and final texts list
clusters_lens = []
final_texts = []

# Process the chunk
threshold = 0.3
sents, vecs = process(txt_content)

# Cluster the sentences
clusters = cluster_text(sents, vecs, threshold)

for cluster in clusters:
    cluster_txt = clean_text(' '.join([sents[i].text for i in cluster]))
    cluster_len = len(cluster_txt)
    
    # Check if the cluster is too short
    if cluster_len < min_token_length:
        continue
    
    # Check if the cluster is too long
    elif cluster_len > max_token_length:
        threshold = 0.5
        sents_div, vecs_div = process(cluster_txt)
        reclusters = cluster_text(sents_div, vecs_div, threshold)
        
        for subcluster in reclusters:
            div_txt = clean_text(' '.join([sents_div[i].text for i in subcluster]))
            div_len = len(div_txt)
            
            if div_len < min_token_length or div_len > max_token_length:
                continue
            
            clusters_lens.append(div_len)
            final_texts.append(div_txt)
            
    else:
        clusters_lens.append(cluster_len)
        final_texts.append(cluster_txt)

In [1]:
# -*- coding:utf-8 -*-
#读取PDF中的图片内容(想要连同图片的介绍文字一同提取下来，还没有实现)

import fitz
import time,os.path,re
time1=time.time()
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams,LTTextBoxHorizontal
from pdfminer.pdfpage import PDFTextExtractionNotAllowed,PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument 


class CPdf2TxtManager():
    # 获取文本内容
    def changePdfToText(self, filePath):
        # 以二进制读模式打开
        file = open(path, 'rb')
        # 用文件对象来创建一个pdf文档分析器
        parser = PDFParser(file)
        # 创建一个PDF文档对象存储文档结构,提供密码初始化,没有就不用传该参数
        doc = PDFDocument(parser, password='')
        # 检查文件是否允许文本提取
        if not doc.is_extractable:
            raise PDFTextExtractionNotAllowed
        # 创建PDf资源管理器来管理共享资源，#caching = False不缓存
        rsrcmgr = PDFResourceManager(caching = False)
        # 创建一个PDF设备对象
        laparams = LAParams()
        # 创建一个PDF页面聚合对象
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        # 创建一个PDF解析器对象
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        # 获得文档的目录(纲要),文档没有纲要会报错
        # PDF文档没有目录时会报：raise PDFNoOutlines pdfminer.pdfdocument.PDFNoOutlines
        # print(doc.get_outlines())
        # 获取page列表
        # print(PDFPage.get_pages(doc))
        # 循环遍历列表，每次处理一个page的内容
        _data =""
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
        # 接受该页面的LTPage对象
        layout = device.get_result()
        # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象
        # 一般包括LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal等等
        for x in layout:
            if hasattr(x, "get_text"):
                fileNames = os.path.splitext(filePath)
                with open(fileNames[0] + '.txt','a+') as f:
                    results = x.get_text()
                    f.write(results + '\n')
                    _data+=results
        # # 如果x是水平文本对象的话
        # if (isinstance(x, LTTextBoxHorizontal)):
        # # print('kkk')
        # text = re.sub(" ", '', x.get_text())
        # if len(text) != 0:
        # _data+=text
        return _data
    
    
    
    # 获取图片
    def pdf2pic(self, path, pic_path):
        t0 = time.perf_counter() # 生成图片初始时间
        checkXO = r"/Type(?= */XObject)" # 使用正则表达式来查找图片
        checkIM = r"/Subtype(?= */Image)"
        doc = fitz.open(path) # 打开pdf文件
        imgcount = 0 # 图片计数
        lenXREF = doc.xref_length() # 获取对象数量长度
        # 打印PDF的信息
        print("文件名:{}, 页数: {}, 对象: {}".format(path, len(doc), lenXREF - 1))
        # 遍历每一个对象
        for i in range(1, lenXREF):
            text = doc.xref_object(i) # 定义对象字符串
            isXObject = re.search(checkXO, text) # 使用正则表达式查看是否是对象
            isImage = re.search(checkIM, text) # 使用正则表达式查看是否是图片
            if not isXObject or not isImage: # 如果不是对象也不是图片，则continue
                continue
            imgcount += 1
            pix = fitz.Pixmap(doc, i) # 生成图像对象
            new_name = "图片{}.png".format(imgcount) # 生成图片的名称
            if pix.n < 5: # 如果pix.n<5,可以直接存为PNG
                pix.save(os.path.join(pic_path, new_name))
            else: # 否则先转换CMYK
                pix0 = fitz.Pixmap(fitz.csRGB, pix)
                pix0.save(os.path.join(pic_path, new_name))
                pix0 = None
                pix = None # 释放资源
                t1 = time.clock() # 图片完成时间
                print("运行时间:{}s".format(t1 - t0))
                print("提取了{}张图片".format(imgcount))
    
    
    
if __name__ == '__main__':
    path = r'./test6.pdf'
    pdf2TxtManager = CPdf2TxtManager()
    df = pdf2TxtManager.changePdfToText(path)
    print(df)
    if not df:
        pic_path = r'./pictures'
    # 创建保存图片的文件夹
    if os.path.exists(pic_path):
        print("文件夹已存在，不必重新创建！")
        pass
    else:
        os.mkdir(pic_path)
    df = pdf2TxtManager.pdf2pic(path,pic_path)
    time2 = time.time()
    print('ok,解析pdf结束!')
    print('总共耗时：' + str(time2 - time1) + 's')


文件夹已存在，不必重新创建！
文件名:./test6.pdf, 页数: 1, 对象: 59
ok,解析pdf结束!
总共耗时：0.1509995460510254s


In [4]:
import re

table_pattern = r'\b(# Table of Contents|# Contents)\b'
figure_pattern = r'\b(Figures\s{1}\d|Figure|#fi)\b'

def get_pages(check_pattern, text):
    pages = text.split('---\n')

    # First check the table of contents
    match = re.search(check_pattern, text, re.IGNORECASE)
    if match:
        matched_pages = []
        for n, page in enumerate(pages):
            if re.search(check_pattern, page, re.IGNORECASE):
                matched_pages.append(n)
        return matched_pages
    return []

def clean_text(text):
    # 匹配从 "Figures\s{1}\d" 开始到换行符之间的所有内容
    pattern = r'(Figures\s{1}\d.*?)\n'
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return cleaned_text

# 示例使用
text = """
Some text here.
Figures 1 This is a figure description.
More text here.
"""

cleaned_text = clean_text(text)
print(cleaned_text)



Some text here.
More text here.

